In [ ]:
# 4_批次质量推断.ipynb
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, norm, beta
from utils import StatisticalAnalyzer

# 模拟批次检测数据
np.random.seed(42)

# 1. 模拟批次参数
true_defect_rate = 0.04  # 真实不良品率
batch_size = 1000  # 批次大小
sample_size = 100   # 抽样数量

# 生成批次数据
batch = np.random.binomial(1, true_defect_rate, batch_size)

# 随机抽样
sample_idx = np.random.choice(batch_size, sample_size, replace=False)
sample = batch[sample_idx]
defective_count = np.sum(sample)

print("=== 批次质量推断 ===")
print(f"批次真实不良品率: {true_defect_rate:.1%}")
print(f"抽样数量: {sample_size}")
print(f"样本中不良品数量: {defective_count}")
print(f"样本不良品率: {defective_count/sample_size:.1%}")

# 2. 抽样分布
plt.figure(figsize=(14, 10))

# 2.1 二项分布PMF
plt.subplot(2, 3, 1)
x_binom = np.arange(0, 20)
pmf_values = binom.pmf(x_binom, n=sample_size, p=true_defect_rate)
plt.bar(x_binom, pmf_values, alpha=0.6)
plt.axvline(defective_count, color='red', linestyle='--', label=f'观察值={defective_count}')
plt.xlabel('不良品数量 X')
plt.ylabel('概率 P(X=x)')
plt.title('二项分布 B(100, p=0.04) 的PMF')
plt.legend()
plt.grid(True, alpha=0.3)

# 2.2 正态近似
plt.subplot(2, 3, 2)
mean_norm = sample_size * true_defect_rate
std_norm = np.sqrt(sample_size * true_defect_rate * (1 - true_defect_rate))
x_norm = np.linspace(0, 20, 1000)
pdf_norm = norm.pdf(x_norm, mean_norm, std_norm)
plt.plot(x_norm, pdf_norm, 'r-', linewidth=2, label='正态近似')
plt.bar(x_binom, pmf_values, alpha=0.3, label='二项分布')
plt.xlabel('不良品数量 X')
plt.ylabel('概率密度')
plt.title('中心极限定理的正态近似')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. 点估计
p_hat = defective_count / sample_size
print(f"\n=== 点估计结果 ===")
print(f"矩估计: p̂ = {p_hat:.4f}")
print(f"极大似然估计: p̂ = {p_hat:.4f}")

# 无偏性模拟
n_simulations = 10000
estimates = []
for _ in range(n_simulations):
    sim_sample = np.random.binomial(sample_size, true_defect_rate)
    estimates.append(sim_sample / sample_size)

bias = np.mean(estimates) - true_defect_rate
mse = np.mean((np.array(estimates) - true_defect_rate)**2)
print(f"\n估计量性质（模拟{format(n_simulations, ',')}次）:")
print(f"偏差: {bias:.6f}")
print(f"均方误差: {mse:.6f}")
print(f"估计量标准差: {np.std(estimates):.6f}")

# 4. 区间估计
# 正态近似区间
z_alpha = norm.ppf(0.975)
se = np.sqrt(p_hat * (1 - p_hat) / sample_size)
ci_normal = (p_hat - z_alpha * se, p_hat + z_alpha * se)

# Wilson区间（更好的小样本性质）
def wilson_ci(x, n, confidence=0.95):
    """Wilson score interval"""
    z = norm.ppf(1 - (1 - confidence) / 2)
    p_hat = x / n
    denominator = 1 + z**2 / n
    center = (p_hat + z**2 / (2 * n)) / denominator
    margin = z * np.sqrt(p_hat * (1 - p_hat) / n + z**2 / (4 * n**2)) / denominator
    return center - margin, center + margin

ci_wilson = wilson_ci(defective_count, sample_size)

print(f"\n=== 区间估计结果 ===")
print(f"95% 置信区间（正态近似）: ({ci_normal[0]:.4f}, {ci_normal[1]:.4f})")
print(f"95% 置信区间（Wilson）: ({ci_wilson[0]:.4f}, {ci_wilson[1]:.4f})")
print(f"区间宽度: {ci_normal[1] - ci_normal[0]:.4f}")

# 4.1 置信区间可视化
plt.subplot(2, 3, 3)
p_values = np.linspace(0, 0.1, 1000)
likelihoods = [binom.pmf(defective_count, sample_size, p) for p in p_values]
max_likelihood = p_values[np.argmax(likelihoods)]

plt.plot(p_values, likelihoods, 'b-', linewidth=2)
plt.axvline(p_hat, color='red', linestyle='--', label=f'MLE = {p_hat:.3f}')
plt.axvline(ci_normal[0], color='green', linestyle=':', label=f'CI下限 = {ci_normal[0]:.3f}')
plt.axvline(ci_normal[1], color='green', linestyle=':', label=f'CI上限 = {ci_normal[1]:.3f}')
plt.fill_between(p_values, 0, likelihoods, 
                 where=(p_values >= ci_normal[0]) & (p_values <= ci_normal[1]), 
                 alpha=0.3, color='green', label='95%置信区间')
plt.xlabel('不良品率 p')
plt.ylabel('似然函数 L(p)')
plt.title('似然函数与置信区间')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. 假设检验
print(f"\n=== 假设检验 ===")
print(f"原假设 H₀: p ≤ 0.03")
print(f"备择假设 H₁: p > 0.03")
print(f"显著性水平 α = 0.05")

test_result = StatisticalAnalyzer.hypothesis_test_proportion(
    defective_count, sample_size, p0=0.03, alpha=0.05, alternative='greater'
)

print(f"检验统计量 z = {test_result['z_statistic']:.3f}")
print(f"P值 = {test_result['p_value']:.4f}")
print(f"是否拒绝H₀: {test_result['reject_H0']}")

# 5.1 检验功效分析
def calculate_power(true_p, n, alpha=0.05, null_p=0.03):
    """计算检验功效"""
    z_alpha = norm.ppf(1 - alpha)
    se_null = np.sqrt(null_p * (1 - null_p) / n)
    critical_value = null_p + z_alpha * se_null
    
    se_true = np.sqrt(true_p * (1 - true_p) / n)
    power = 1 - norm.cdf((critical_value - true_p) / se_true)
    return power

true_rates = np.linspace(0.01, 0.08, 15)
powers = [calculate_power(p, sample_size) for p in true_rates]

plt.subplot(2, 3, 4)
plt.plot(true_rates, powers, 'o-', linewidth=2)
plt.axvline(0.03, color='red', linestyle='--', label='H₀: p=0.03')
plt.axvline(true_defect_rate, color='green', linestyle='--', label=f'真实p={true_defect_rate:.3f}')
plt.axhline(0.8, color='gray', linestyle=':', label='80%功效')
plt.xlabel('真实不良品率 p')
plt.ylabel('检验功效')
plt.title('假设检验的功效曲线')
plt.legend()
plt.grid(True, alpha=0.3)

# 5.2 P值分布（模拟）
plt.subplot(2, 3, 5)
n_sim = 1000
p_values_sim = []
for _ in range(n_sim):
    sim_defective = np.random.binomial(sample_size, true_defect_rate)
    test_sim = StatisticalAnalyzer.hypothesis_test_proportion(
        sim_defective, sample_size, p0=0.03, alpha=0.05, alternative='greater'
    )
    p_values_sim.append(test_sim['p_value'])

plt.hist(p_values_sim, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(0.05, color='red', linestyle='--', linewidth=2, label='α=0.05')
plt.xlabel('P值')
plt.ylabel('频数')
plt.title('P值分布（模拟1000次）')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. 样本量对置信区间的影响
plt.subplot(2, 3, 6)
sample_sizes = np.arange(30, 501, 10)
ci_widths = []

for n in sample_sizes:
    se = np.sqrt(p_hat * (1 - p_hat) / n)
    width = 2 * z_alpha * se
    ci_widths.append(width)

plt.plot(sample_sizes, ci_widths, 'b-', linewidth=2)
plt.axhline(0.05, color='red', linestyle='--', label='目标宽度=0.05')
plt.axvline(sample_size, color='green', linestyle=':', label=f'n={sample_size}')
plt.xlabel('样本量 n')
plt.ylabel('置信区间宽度')
plt.title('样本量对区间精度的影响')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('批次质量推断.png', dpi=300, bbox_inches='tight')
plt.show()

# 保存结果
results_part4 = {
    'defective_count': defective_count,
    'p_hat': p_hat,
    'ci_normal': ci_normal,
    'test_result': test_result,
    'sample_size': sample_size
}